# Opening and cleaning out ephys data

Note: Use L1imag/formation environment.

## Setup everything

### Import packages

In [1]:
import quantities as pq
import numpy as np
import neo
from pathlib import Path
import fnmatch
import os
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
import scipy
from scipy import interpolate
from scipy import fftpack
from scipy import signal
%matplotlib widget
from ipyfilechooser import FileChooser

from ephyviewer import mkQApp, MainViewer, TraceViewer

# add the Contrib dir that contains all tools developped by MB : mbTools.py
#sys.path.append(os.path.join(os.path.dirname(sys.path[0]),'python'))
#print(os.path.join(os.path.dirname(sys.path[0]),'python'))
import mbTools

### Choose experiment

In [2]:
dpath = None
%store -r dpath
print(dpath)
try:
    theExpe = mbTools.experiment(dpath)
except:
    theExpe = mbTools.experiment()

/Volumes/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording_ABmodified/RedLinesOK/session1/OpenEphys/


FileChooser(path='/Volumes/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording_ABmodified/…

## Load data
### Load timestamps from continuous recording

Note here the synchronised timestamps are not loaded. Attempt is made next cell but there is a bug because the files are of different lengths. 
What needs to be done is to understand the exact content of these files to then load them properly.

In [3]:
Ephys_rec_stamps = theExpe.loadRecording_TimeStamps()


/Users/mb/Documents/Syntuitio/AudreyHay/PlanB/python


### Load timestamps for miniscope frames and laser (TTL in)


In [4]:

TTL_stamps = theExpe.loadTTL_TimeStamps()
print(TTL_stamps)


/Users/mb/Documents/Syntuitio/AudreyHay/PlanB/python
[]


### Load recordings

#### Merge files

In [5]:
theExpe.numchannels=32
All = theExpe.loadLFP()

NPY file loaded, with 31 channels and 940555 datapoint


### Distribute channels.
> **WARNING:** this has to be adjusted for every mouse
defineMap function takes as the second argument an array of dict with every cannal number corresponding to the relevant structure with a status: 0 for unused, 1 for floating electrode, 1 and 2 for differential

In [9]:
sampling_rate = 25000 # Hz

# Distribute
theExpe.defineMap('EMG',[dict(canal = 10, status=1)])
theExpe.defineMap('PFC',[dict(canal = 17, status=1)])
theExpe.defineMap('S1',[dict(canal = 22, status=1),
                         dict(canal = 23, status=2)])
theExpe.defineMap('CA1',[dict(canal = 20, status=1),
                         dict(canal = 19, status=2)])


# Stack back
#combined = theExpe.combineStructures(['EMG', 'PFC', 'S1', 'CA1'])
combined = theExpe.combineStructures(theExpe.channelsMap)

coords = {
    'brain_areas' : np.array(['EMG', 'PFC', "S1", "CA1"]),
    'duration_rec' : np.arange(0, combined.shape[0]/sampling_rate, 1/sampling_rate)
}

# Put in xarray
xrCombined = xr.DataArray(coords=coords, dims=['duration_rec', 'brain_areas'])
xrCombined.loc[:,:]  = combined

# Save datas
combinedFN = os.path.join(dpath,'RawDataChannelExtracted.npy')
np.save(combinedFN, combined)


EMG -> [{'canal': 10, 'status': 1}]
Getting floating signal of channel 10 for EMG
PFC -> [{'canal': 17, 'status': 1}]
Getting floating signal of channel 17 for PFC
S1 -> [{'canal': 22, 'status': 1}, {'canal': 23, 'status': 2}]
Getting differential signal of channel 23 - channel 22 for S1
CA1 -> [{'canal': 20, 'status': 1}, {'canal': 19, 'status': 2}]
Getting differential signal of channel 19 - channel 20 for CA1
/Volumes/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording_ABmodified/RedLinesOK/session1/OpenEphys/RawDataChannelExtracted.npy


#  Downsample all signals to 1 kz.


In [ ]:
combined = All[:100000]
new_sampling_rate = 1000 # Hz

newLen = int(combined.shape[0] * new_sampling_rate / sampling_rate)
print(combined.shape[0], newLen)
combinedDS = signal.resample(combined, newLen, axis = 0)
#combinedDS = signal.decimate(combined, 25, axis = 0)
print('resampled', combinedDS.shape)
combinedDSFN = os.path.join(dpath,'RawDataChannelExtractedDS.npy')
np.save(combinedDSFN, combinedDS)
print(f"the downsampled file is saved at : {combinedDSFN}")

## End of notebook. 
Data is cleaned up and saved. Data processing for different cortical areas on specific notebooks. Next step is WakeRemoving notebook.

Below is for quick filtering, plotting and visualisation to assess data quality. Filtering for data processing is done again in specific notebooks.

Data quality is assessed on a Sample, whose value has to be attributed on the initial cell.

## Filtering 

SWR: 120 - 200 Hz

In [ ]:
Sample = combinedDS

f_CA1 = Sample[:, 3].copy()

# Paramètres de notre filtre :
f_lowcut = 120.
f_hicut = 200.
fs = new_sampling_rate
nyq = 0.5 * fs
N = 6                 # Ordre du filtre
Wn = [f_lowcut/nyq,f_hicut/nyq]  # Nyquist frequency fraction
print(Wn)

# Création du filtre :
b, a = scipy.signal.butter(N, Wn, 'band')
filt_SWR_CA1 = scipy.signal.filtfilt(b, a, f_CA1)

times = np.arange(0, f_CA1.size/new_sampling_rate, 1./new_sampling_rate)



Spindles: 8 - 16 Hz

In [ ]:
f_PFC = Sample[:, 1].copy()
f_S1 = Sample[:, 2].copy()

# Paramètres de notre filtre :
f_lowcut = 10.
f_hicut = 16.
Wn = [f_lowcut/nyq,f_hicut/nyq]  # Nyquist frequency fraction
N = 4
# Création du filtre :
b, a = scipy.signal.butter(N, Wn, 'band')
filt_Spind_PFC = scipy.signal.filtfilt(b, a, f_PFC)
filt_Spind_S1 = scipy.signal.filtfilt(b, a, f_S1)

# # Calcul de la reponse en fréquence du filtre
# w, h = signal.freqz(b, a)

# # Tracé de la réponse en fréquence du filtre
# fig, ax = plt.subplots(figsize=(8,5)) 

# ax.plot(0.5*fs*w/np.pi, np.abs(h), 'b')

# ax.set_xlabel('frequency [Hz]')
# ax.set_ylabel('Amplitude [dB]')
# ax.grid(which='both', axis='both')

FFT display

In [ ]:
# Calcul du spectre
f, Pxx_den = signal.welch(f_CA1, fs, nperseg=1024)

# Tracé
fig, ax = plt.subplots(figsize=(10,5)) 
ax.semilogy(f, Pxx_den)   #  plot with log scaling on the y axis
ax.set_xlabel('frequency [Hz]')
ax.set_ylabel('PSD [V**2/Hz]')

In [ ]:
plt.close('all')

# Display. 

Can massively be improved: with Matplotlib

In [ ]:
times_sliced = times[000:200000]
filt_to_display = filt_SWR_CA1[000:200000]-1000
f_CA1_sliced = f_CA1[0000:200000]/2
combined = np.stack((f_CA1_sliced, filt_to_display), axis = 1)
# Tracé du signal filtré

fig, ax = plt.subplots(1,1, figsize=(15,5), layout='constrained') 
ax.plot(times_sliced, combined, 'r')
ax.set_xlabel('Temps [sec]')
ax.set_ylabel('Amplitude')


With ephyviewer. 

In [ ]:
# prepare numpy array for ephyviewer

filt_SWR_CA1_sliced= filt_SWR_CA1[000:200000, np.newaxis]
filt_Spind_PFC_sliced= filt_Spind_PFC[000:200000, np.newaxis]
filt_Spind_S1_sliced= filt_Spind_S1[000:200000, np.newaxis]
filt_sliced = Sample[0000:200000,:]
combined2 = filt_sliced[:,0:2].copy()
intf_sliced = filt_sliced[:,2]
intf_sliced = intf_sliced[:, np.newaxis]
combined2 = np.append(combined2, filt_Spind_PFC_sliced, axis=1)
combined2 = np.append(combined2, intf_sliced, axis=1)
combined2 = np.append(combined2, filt_Spind_S1_sliced, axis=1)
intf_sliced = filt_sliced[:,3]
intf_sliced = intf_sliced[:, np.newaxis]
combined2 = np.append(combined2, intf_sliced, axis=1)
combined2 = np.append(combined2, filt_SWR_CA1_sliced, axis = 1)


In [ ]:
%gui qt 
app = mkQApp()

sample_rate = 1000.
t_start = 0.

#Create the main window that can contain several viewers
win = MainViewer()
view1 = TraceViewer.from_numpy(combined2, sample_rate, t_start, 'Signals')
win.add_view(view1)

#Parameters can be set in script
view1.params['display_labels'] = True
view1.params['scale_mode'] = 'same_for_all'
view1.auto_scale()

#Run
win.show()